In [ ]:
import pandas as pd
from datetime import datetime
import ast
import gzip


In [ ]:
with gzip.open('datasets/user_items_proc.csv.gz', 'rb') as f:
    user_items_proc = pd.read_csv(f, encoding='utf-8')
    
with gzip.open('datasets/users_items.csv.gz', 'rb') as f:
    users_items = pd.read_csv(f, encoding='utf-8')
    
steam_games = pd.read_csv('./datasets/steam_games.csv', parse_dates=['release_date'])
user_review = pd.read_csv('./datasets/user_reviews.csv')

In [ ]:
diner_user = pd.merge(user_items_proc[['user_id','items_count','item_name']],steam_games[['app_name','price']],left_on='item_name',right_on='app_name',how='left')

In [48]:
users_items = diner_user.groupby('user_id').agg(
    {
       'items_count':'sum',
        'price':'sum'
    }
).reset_index()

recomendaciones_true = user_review.groupby('user_id')['recommend'].sum()
total_registros_por_usuario = user_review.groupby('user_id')['recommend'].count()
porcentaje_recomendaciones_true = (recomendaciones_true / total_registros_por_usuario) * 100
users_true_reviews = pd.DataFrame({
    'user_id': porcentaje_recomendaciones_true.index,
    'porcentaje_recomendaciones_true': porcentaje_recomendaciones_true.values
})
items_reviews_users = pd.merge(users_items,users_true_reviews,on='user_id',how='inner')
items_reviews_users.to_csv('./datasets/items_reviews_users.csv')

In [50]:
def userdata(User_id:str):
    items_reviews_users = pd.read_csv('datasets/items_reviews_users.csv')
    user = items_reviews_users[items_reviews_users['user_id'].str.lower() == User_id.lower()]
    return {'Usuario X':user['user_id'].iloc[0],
            'Dinero gastado':f'{round(user["price"].iloc[0],2)} USD',
            f'% de recomendación':f'{user["porcentaje_recomendaciones_true"].iloc[0]} %',
            'cantidad de items': user['items_count'].iloc[0]                   
            } 
    


In [ ]:
def UserForGenre(genero:str):
    genero = genero.lower()
    steam_games_cop = steam_games
    steam_games_cop.columns = steam_games.columns.str.lower()
    generos = steam_games.drop(columns=['app_name','release_date','specs','price','id','developer','year']).columns
    if genero not in generos:
        return "No existe ese género"
    # FALTA ARREGLAR QUE EL GENERO PUEDA SER INGRESADO CON MAYUSCULAS O MINUSCULAS    
    usuarios_games = pd.merge(users_items_proc,steam_games,left_on='item_name',right_on='app_name',how='inner')
    usuarios_games = usuarios_games[usuarios_games[genero]==1]
    user_max_hor = usuarios_games.groupby(['user_id'])['playtime_forever'].sum().reset_index()
    user_max_hora = user_max_hor.loc[user_max_hor['playtime_forever'].idxmax(), 'user_id']
    usuario_hora = usuarios_games[usuarios_games['user_id'] == user_max_hora].groupby('year')['playtime_forever'].sum().reset_index()
    lista_resultados = []
    for index, row in usuario_hora.iterrows():
        if row['playtime_forever'] == 0.0:
            continue
            
        diccionario = {
            'Año':int(row['year']),
            'Horas': int(row['playtime_forever'])
        }
        lista_resultados.append(diccionario)
        
    dic = {'Usuario ':user_max_hora,'Horas Jugadas':lista_resultados}
        
    return dic
    

In [ ]:
UserForGenre('action')

In [ ]:
user_review

In [ ]:
def best_developer_year_func(año:str):
    func_4 = pd.merge(user_review,steam_games,left_on='item_id',right_on='id',how='inner')
    func_4 = func_4[func_4['Year'] == año]
    mejores_dev = func_4.groupby('developer')['recommend'].sum().reset_index().sort_values(by='recommend',ascending=False)
    if mejores_dev.empty:
        return 'No se enocntraron reviews para items que hayan salido ese año'
    else:
        puesto1 = mejores_dev.iloc[0][0]
        puesto2 = mejores_dev.iloc[1][0]
        puesto3 = mejores_dev.iloc[2][0]
        puestos = {"Puesto 1": puesto1, "Puesto 2": puesto2, "Puesto 3": puesto3}
        return puestos

In [ ]:
best_developer_year_func(2017)

In [ ]:
user_review['posted'].iloc[0].split(',')[-1].strip(' .')

In [ ]:
def developer_rec_func(desarrolladora:str):
    func_5 = pd.merge(user_review,steam_games,left_on='item_id',right_on='id',how='inner')
    func_5 = func_5[func_5['developer'].str.lower() == desarrolladora.lower()]
    if func_5.empty:
        return 'No se enocntraron reviews para items que hayan salido ese año'
    else:
        true_value = func_5[func_5['recommend']==True]['recommend'].count()
        false_value = func_5[func_5['recommend']==False]['recommend'].count()
        return {desarrolladora:[f'Nevative = {false_value}',f'Positive = {true_value}']}

In [ ]:
developer_rec_func('Tripwire Interactive')

In [ ]:
steam_games

In [ ]:
steam_games = pd.read_csv('./datasets/steam_games.csv')
    

steam_games.dropna(subset=['Year'],inplace=True)
steam_games['Year'] = steam_games['Year'].astype(int)

In [ ]:
developer_func('Poolians.com')

In [ ]:
steam_games['developer'].values